In [134]:
import random
import json
import time
from datetime import datetime as dt, timedelta
import sys
import ProcessMining_API as IPM
import pandas as pd

def join_and_cleanup(df1, df2, cols, suffix, suffix_for):
    # suffix_for = 'left' or 'right'
    print(df1)
    print(df2)
    if (suffix_for == 'left'): suffixes = (suffix, '')
    else: suffixes = ('', suffix)
    df = df1.merge(df2, how='left', on=cols, indicator='alert_is_present_on', suffixes=suffixes)
    # remove columns (duplicated) from history
    kept_cols = []
    for col in df.columns:
        if (suffix not in col):
            kept_cols.append(col)
    return df[kept_cols]

class widgetAlerts():
    def __init__(self, config): # config is a json with the widget properties
        self.config = config
        self.dashboardName = config['dashboard_name']
        self.widgetId = config['widget_id']
        self.historyFileName = 'alert_history_' + self.dashboardName + '_' + self.widgetId +'.csv'
        self.summaryFileName = 'alert_summary_' + self.dashboardName + '_' + self.widgetId +'.csv'


    def loadHistory(self):
        # Load alert history for this widget
        try: 
            self.history_df = pd.read_csv(self.historyFileName, dtype=str)
        except:
            # First call: Create the historical CSV from scratch at the end
            self.history_df = pd.DataFrame()

    def loadSummary(self):
        # Load summary file for this widget
        try: 
            self.summary_df = pd.read_csv(self.summaryFileName)
        except:
            # First call: Create the summary df 
            self.summary_df = pd.DataFrame()

    def loadWidgetData(self):
        IPM.ws_post_sign(self.config)
        widgetVals = IPM.ws_get_widget_values(self.config)
        if widgetVals: self.widget_df = pd.DataFrame(widgetVals)
        else: 
            print('ERROR: Dashboard or Widget not found')
            self.widget_df

    def setMatchingCols(self, cols):
        # Get the widget columns used to match the alerts
        # Do the merge with the columns mentionned in the configuration as an array
        if cols == 0 or cols == '' : self.matchingColumns = self.widget_df.columns[0]
        else:
            self.matchingColumns = []
            for i in range(len(self.widget_df.columns)):
                if (i in cols):
                    self.matchingColumns.append(self.widget_df.columns[i])
                                                
        print("Matching columns used for existing alerts: %s" % self.matchingColumns)
    
    def updateAlertsFirstTime(self):
        if len(self.widget_df) == 0:
            print('Empty widget and no history yet. No data to generate')
            self.final_df = 0
            self.summary_df = 0
        else: #first time execution
            self.final_df = self.widget_df.copy()
            self.final_df['alert_status'] = 'NEW'
            self.final_df['alert_creation_date'] = dt.now().isoformat()
            self.final_df['alert_closed_date'] = ''
            self.summary_df = pd.DataFrame([{
                'update_date': dt.now().isoformat(),
                'new': len(self.widget_df),
                'pending': 0,
                'closed': 0,
                'new_to_pending': 0,
                'new_to_closed': 0,
                'pending_to_closed': 0,
                'pending_to_pending': 0,
                'any_to_closed': 0,
                'progression_rate': 0
            }])

    def updateAlerts(self):
        self.loadHistory()
        self.loadSummary()
        self.loadWidgetData()
        self.setMatchingCols(self.config['alert_matching_columns'])

        if len(self.history_df) == 0: # No history yet
            self.updateAlertsFirstTime()
        else:
            self.final_df = self.history_df[self.history_df.alert_status == 'CLOSED'] # we don't touch CLOSED alerts
            # Add a new summary row
            self.summary_df.loc[len(self.summary_df)] = { 
                'update_date': dt.now().isoformat(),
                'new': 0,
                'pending': 0,
                'closed': self.summary_df.loc[len(self.summary_df)-1, 'closed'],
                'new_to_pending': 0,
                'new_to_closed': 0,
                'pending_to_closed': 0,
                'pending_to_pending': 0,
                'any_to_closed': 0,
                'progression_rate': 0
            }
            if len(self.widget_df) == 0: # no more alerts, close all NEW and PENDING from history
                self.closeAllAlerts()
            else:
                self.manageNewAlerts()
                self.manageExistingAlerts()

    def closeAllAlerts(self):
        new_to_close_df = self.history_df[self.history_df.alert_status == 'NEW']
        new_to_close_df.alert_status = 'CLOSED'
        new_to_close_df.alert_closed_date = dt.now().isoformat()
        pending_to_close_df = self.history_df[self.history_df.alert_status == 'PENDING']
        pending_to_close_df.alert_status = 'CLOSED'
        pending_to_close_df.alert_closed_date = dt.now().isoformat()
        self.final_df = self.final_df.concate([self.final_df, new_to_close_df, pending_to_close_df])
        self.summary_df.loc[len(self.summary_df) - 1, 'closed'] = len(self.history_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'new_to_closed'] = len(new_to_close_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'pending_to_closed'] = len(pending_to_close_df)

    def manageNewAlerts(self):
        histo_not_closed_df = self.history_df[self.history_df.alert_status != 'CLOSED']
        widget_new_df = join_and_cleanup(self.widget_df, histo_not_closed_df, self.matchingColumns, "_suffixFromHistory", 'right')
        # Alerts in the widget with 'exist'==left_only are NEW
        widget_new_df = widget_new_df[widget_new_df.alert_is_present_on == 'left_only']
        widget_new_df.alert_status = 'NEW'
        widget_new_df.alert_creation_date = dt.now().isoformat()
        widget_new_df.alert_closed_date = ''
        self.final_df = pd.concat([self.final_df, widget_new_df.drop(columns=['alert_is_present_on'])])
        self.summary_df.loc[len(self.summary_df) - 1, 'new'] = len(widget_new_df)

    def manageExistingAlerts(self):
        histo_not_closed_df = self.history_df[self.history_df.alert_status != 'CLOSED']
        histo_not_closed_df = join_and_cleanup(histo_not_closed_df, self.widget_df, self.matchingColumns, "_suffixFromHistory", 'left')

        # NEW to PENDING  
        histo_new_to_pending_df = histo_not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="both"')
        self.summary_df.loc[len(self.summary_df) - 1, 'new_to_pending'] = len(histo_new_to_pending_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'pending'] = len(histo_new_to_pending_df)
        if (len(histo_new_to_pending_df)):
            histo_new_to_pending_df.alert_status = 'PENDING'
            self.final_df = pd.concat([self.final_df, histo_new_to_pending_df.drop(columns=['alert_is_present_on'])])

        # NEW to CLOSED
        histo_new_to_close_df = histo_not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="left_only"')
        self.summary_df.loc[len(self.summary_df) - 1, 'new_to_closed'] = len(histo_new_to_close_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'closed'] += len(histo_new_to_close_df)
        if (len(histo_new_to_close_df)):
            histo_new_to_close_df.alert_status = 'CLOSED'
            histo_new_to_close_df.alert_closed_date = dt.now().isoformat()
            self.final_df = pd.concat([self.final_df, histo_new_to_close_df.drop(columns=['alert_is_present_on'])])

        # PENDING to PENDING
        histo_pending_to_pending_df = histo_not_closed_df.query('alert_status=="PENDING" & alert_is_present_on=="both"')
        self.summary_df.loc[len(self.summary_df) - 1, 'new_to_pending'] = len(histo_pending_to_pending_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'pending'] += len(histo_pending_to_pending_df)
        if (len(histo_pending_to_pending_df)):
            histo_pending_to_pending_df.alert_status = 'PENDING'
            self.final_df = pd.concat([self.final_df, histo_pending_to_pending_df.drop(columns=['alert_is_present_on'])])    

        # PENDING to CLOSED
        pending_to_closed_df = histo_not_closed_df.query('alert_status=="NEW" & alert_is_present_on=="left_only"')
        self.summary_df.loc[len(self.summary_df) - 1, 'pending_to_closed'] = len(pending_to_closed_df)
        self.summary_df.loc[len(self.summary_df) - 1, 'closed'] += len(pending_to_closed_df)
        if (len(pending_to_closed_df)):
            pending_to_closed_df.alert_status = 'CLOSED'
            pending_to_closed_df.alert_closed_date = dt.now().isoformat()
            self.final_df = pd.concat([self.final_df, pending_to_closed_df.drop(columns=['alert_is_present_on'])])         

    def saveHistoryFile(self):
        if (len(self.final_df)):
            self.final_df.to_csv(self.historyFileName, index=None)
    
    def saveSummaryFile(self):
        if (len(self.summary_df)):
            self.summary_df.to_csv(self.summaryFileName, index=None)     



In [135]:
mywidget = widgetAlerts({
    "url":"https://pharoses1.fyre.ibm.com",
    "user_id": "task.miner",
    "api_key":"8a5kga87eqvd1180",
    "project_key": "procure-to-pay",
    "org_key": "",
    "dashboard_name": "Alerts Dashboard",
    "widget_id": "alerts-widget-1",
    "alert_matching_columns": [0,1,2,3]
})


In [137]:
mywidget.loadHistory()
mywidget.loadSummary()
mywidget.loadWidgetData()



....searching dashboard_id


In [138]:
mywidget.setMatchingCols(mywidget.config['alert_matching_columns'])
mywidget.matchingColumns

Matching columns used for existing alerts: ['attr-process', 'InvoiceID', 'Activity', 'Date']


['attr-process', 'InvoiceID', 'Activity', 'Date']

In [139]:
histo_not_closed_df = mywidget.history_df[mywidget.history_df.alert_status != 'CLOSED']
df = histo_not_closed_df.merge(mywidget.widget_df, on=mywidget.matchingColumns, how='left', indicator='toto')
df

,attr-process,InvoiceID,Activity,Date,Resource_x,Vendor_x,Amount_x,alert_status,alert_creation_date,alert_closed_date,Resource_y,Vendor_y,Amount_y,toto
0,19593,3118002701_2018_IT10,Invoice Changed: Withholding Tax,1.544440103E12,DAM2A04,VND04049,688.76,NEW,2024-02-18T10:26:48.506701,NaN,DAM2A04,VND04049,688.76,both
1,12268,2018020502_2018_IT10,Invoice Changed: Withholding Tax,1.539604055E12,USR00357,VND05417,-7199.22,NEW,2024-02-18T10:26:48.506701,NaN,USR00357,VND05417,-7199.22,both
2,12266,2018020500_2018_IT10,Invoice Changed: Withholding Tax,1.539603975E12,USR00357,VND05417,7199.22,NEW,2024-02-18T10:26:48.506701,NaN,USR00357,VND05417,7199.22,both
